In [1]:
import urllib.request
import numpy as np
from utils import utils

### Load further embeddings

In [ ]:
embedding_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors.txt"
vocab_original, w2id_original, embedding_original = utils.read_embedding(embedding_url)
vocab, w2id, embedding = utils.restrict_vocab(vocab_original, w2id_original, embedding_original)

In [2]:
embedding_gn_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors300.txt"
vocab_gn_original, w2id_gn_original, embedding_gn_original = utils.read_embedding(embedding_gn_url)
vocab_gn, w2id_gn, embedding_gn = utils.restrict_vocab(vocab_gn_original, w2id_gn_original, embedding_gn_original)

In [3]:
vocab_gn_a = vocab_gn
w2id_gn_a = w2id_gn
embedding_gn_a = utils.debias_gn(embedding_gn)

In [ ]:
embedding_hd_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors_hd.txt"
vocab_hd_original, w2id_hd_original, embedding_hd_original = utils.read_embedding(embedding_hd_url)
vocab_hd, w2id_hd, embedding_hd = restrict_vocab(vocab_hd_original, w2id_hd_original, embedding_hd_original)

In [ ]:
embedding_hd_a_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors_hd_a.txt"
vocab_hd_a_original, w2id_hd_a_original, embedding_hd_a_original = utils.read_embedding(embedding_hd_a_url)
vocab_hd_a, w2id_hd_a, embedding_hd_a = restrict_vocab(vocab_hd_a_original, w2id_hd_a_original, embedding_hd_a_original)

In [ ]:
embedding_gp_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/gp_glove.txt"
vocab_gp_original, w2id_gp_original, embedding_gp_original = utils.read_embedding(embedding_gp_url, skip_first = True)
vocab_gp, w2id_gp, embedding_gp = restrict_vocab(vocab_gp_original, w2id_gp_original, embedding_gp_original)

In [ ]:
embedding_gp_gn_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/gp_gn_glove.txt"
vocab_gp_gn_original, w2id_gp_gn_original, embedding_gp_gn_original = utils.read_embedding(embedding_gp_gn_url, skip_first = True)
vocab_gp_gn, w2id_gp_gn, embedding_gp_gn = restrict_vocab(vocab_gp_gn_original, w2id_gp_gn_original, embedding_gp_gn_original)

### Word Embedding Association Test
Implementation taken from "https://github.com/shivaomrani/HumanBiasInSemantics". 
With minor adjustments such as variable names for readability, see file "weat.py".

In [1]:
from word_association_test import weat
# auf embedding zugreifen, wenn wir das als .py haben

In [2]:
# Career and family
# Change from Bill to Tom as in paper to avoid ambiguity
male_names = ["john", "paul", "mike", "kevin", "steve", "greg", "jeff", "tom"]
female_names = ["amy", "joan", "lisa", "sarah", "diana", "kate", "ann", "donna"]
career_attributes = ["executive", "management", "professional", "corporation", "salary", "office", "business", "career"]
family_attributes = ["home", "parents", "children", "family", "cousins", "marriage", "wedding", "relatives"]

In [7]:
# Math and arts
math_words = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
arts_words1 = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
male_attributes1 = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
female_attributes1 = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]

In [8]:
# Science and arts
science_words = ["science", "technology", "pyhsics", "chemistry", "einstein", "nasa", "experiment", "astronomy"]
arts_words2 = ["poetry", "art", "shakespeare", "dance", "literature", "novel", "symphony", "drama"]
male_attributes2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
female_attributes2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "her"]

In [4]:
iterations = 100000
wea_test = weat(male_names, female_names, career_attributes, family_attributes, iterations, embedding, w2id)
pvalue, effect_size, sd = wea_test.getPValueAndEffect()
print("p-value: ", pvalue)
print("effect size: ", effect_size)
print("standard deviation: ", sd)

NameError: name 'embedding' is not defined

### Word Analogy
The word analogy task is to find a word *D* such that "*A* is to *B* as *C* is to *D*". Wang et al. (2020) have evaluated all non-debiased and debiased embeddings on the MSR word analogy task (Mikolov et al., 2013 Ling) as well as on a second Google word analogy dataset (Mikolov et al. 2013 Eff). The evaluation metric is the percentage of questions for which the correct answer is assigned the maximum score by the algorithm. The analogy task is used to show whether a debiasing method is capable of preserving desired distance relations between words.

In [23]:
from word_analogy import analogy_tasks as ana

ModuleNotFoundError: No module named 'web'